# Collision Avoidance - Live Demo（避障-现场演示）

在这 Notebook中，我们将使用训练好的模型来检测小车是 ``free`` 还是 ``blocked``，以便在小车上实现避免碰撞的行为。

## 载入训练好的模型

假设您已经按照培训笔记本中的说明将 ``best_model.pth`` 下载到您的工作站。现在，你应该把这个模型上传（upload）到这个笔记本的Jupyter Lab工具的目录。完成后，这个目录中应该有一个名为 ``best_model.pth`` 文件。

> 在调用下一个单元格之前，请确保文件已完全上载

执行下面的代码来初始化 PyTorch 模型。这与training 的 notebook 看起来颇为类似。

In [ ]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

接下来，从上传的 ``best_model.pth`` 文件中加载经过训练的权重

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

目前，模型权重位于CPU内存上，执行下面的代码传输到GPU设备。

In [ ]:
device = torch.device('cuda')
model = model.to(device)

### 创 建预处理函数

我们现在已经加载了模型，但有一个小问题。我们训练模型的格式与相机的格式不完全匹配。要做到这一点，我们需要做一些 *预处理* ，包括以下步骤

1. 从 BGR 转换为 RGB
2. 从 HWC 布局转换为 CHW 布局
3. 使用与训练期间相同的参数进行规格化（我们的相机提供[0，255]范围内的值和[0，1]范围内的训练加载图像，因此我们需要缩放 255.0
4. 将数据从CPU存储器传输到GPU存储器
5. 添加批次维度

In [ ]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

很好，我们已经定义了我们的预处理函数，它可以将图像从摄像机格式转换为神经网络输入格式。

现在，让我们开始展示我们的相机。你现在应该已经很熟悉了。我们还将创建一个滑块来显示小车被阻挡的可能性。

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

我们还要创建小车实例，来驱动电机。

In [ ]:
from jetbot import Robot

robot = Robot()

接下来，我们将创建一个函数，每当相机的值发生变化时，该函数将被调用。此函数将执行以下步骤

1. 预处理摄像机图像
2. 执行神经网络
3. 当神经网络输出显示我们被阻塞时，我们会左转，否则我们继续前进。

In [ ]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # 我们应用 `softmax` 函数来规范化输出向量，使其和为1（这使其成为概率分布）
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5:
        robot.forward(0.4)
    else:
        robot.left(0.4)
    
    time.sleep(0.001)
        
update({'new': camera.value})  # 我们调用函数一次来初始化
robot.stop() 

太酷了！我们已经创建了我们的神经网络执行函数，但现在我们需要将它附加到相机上进行处理，通过``observe`` 来实现这一点。

>警告：此代码将使小车移动 ！请确保有足够空间。避免碰撞程序应该已经启动，但是神经网络只和它训练的数据一样好！

In [ ]:
camera.observe(update, names='value')  # 附加到相机的 “update” 功能将更新 traitlet 的 "value"

一旦这个附件功能接入小车，它就会在每一个相机帧更新后产生新的命令。先把小车放在地上，看看遇到障碍物时会做什么？

通过执行下面的代码取消附加此回调，就能停止整个动作

In [ ]:
camera.unobserve(update, names='value')
robot.stop()

如果你想让机器人在不向浏览器传输视频的情况下运行。您可以按如下方式解除相机的链接。

In [ ]:
camera_link.unlink()  # don't stream to browser (will still run camera)

要继续流式处理，请调用以下命令。

In [ ]:
camera_link.link()  # stream to browser (wont run camera)

### Conclusion

这就是现场演示！希望你玩得开心，你的小车聪明地避免了碰撞！
如果你的小车不能很好地避免碰撞，试着找出它失败的地方。好处是我们可以为这些失败场景收集更多的数据，机器人应该会变得更好：）